
2. O Conway’s Game of Life é um exemplo bastante conhecido de um autómato celular . Neste problema vamos modificar as regras do autómato da seguinte forma

    1. O espaço de estados é finito definido por uma grelha de células booleanas (morta=0/viva=1) de dimensão $\,N\times N\,$ (com $N>3$) identificadas por índices $\,(i,j)\in \{1..N\}$.  Estas $\;N^2\;$ células são aqui referidas como “normais”. 
        No estado inicial todas as células normais estão mortas excepto  um quadrado $\,3\times 3\,$, designado por “centro”, aleatoriamente posicionado formado apenas por células vivas.
    2. Adicionalmente existem $\,2\,N+1\,$ “células da borda” que correspondem a um dos índices, $i$ ou $j$, ser zero. As células da borda têm valores constantes que, no estado inicial, são gerados aleatoriamente com uma probabilidade $\,\rho\,$ de estarem vivas.
    3. As células normais o autómato modificam o estado de acordo com a regra “B3/S23”: i.e. a célula nasce (passa de $0$ a $1$) se tem exatamente 3 vizinhos vivos e sobrevive (mantém-se viva) se o número de vizinhos vivos é 2 ou 3, caso contrário morre ou continua morta.

A célula $\;(i_0,j_0)\;$e $\,(i_1,j_1)\,$ são vizinhas sse $\;(i_0-i_1=\pm 1) \,\lor\, (j_0-j_1 = \pm 1)$

Pretende-se:

Construir uma máquina de estados finita que represente este autómato; são parâmetros do problema os parâmetros $\,N,\rho\,$ e a posição do  “centro”.

Verificar se se conseguem provar as seguintes propriedades:
   1. Todos os estados acessíveis contém pelo menos uma célula viva.
   2. Toda a célula normal está viva pelo menos uma vez em algum estado acessível.


In [1]:
#from pysmt.shortcuts import *
from z3 import *

In [2]:
def declare(t, N):
    state = {}
    state['pc'] = Int('pc'+str(t))
    state['pos'] = {}
    for i in range(N):
        for j in range(N):
            state['pos'][f'({i},{j})'] = BitVec(f'pos({i},{j})'+str(t), 5)


    return state

In [3]:
'''
from pyprobs import Probability as pr
probabilidade de sair True = 25%
pr.prob("25%", num=5)
[False, False, True, False, False]
'''
from pyprobs import Probability as pr
from random import randint

def init(state, prob, N):
    init_state = {}

    #center_square = (randint(1, N-2), randint(1, N-2))
    
    center_square = (2,2)
    
    # colocar o quadrado a True
    for i in range(center_square[0]-1, center_square[0]+2):
        for j in range(center_square[1]-1, center_square[1]+2):
            init_state[(i,j)] = (state['pos'][f'({i},{j})'] == BitVecVal(1,5))
    
    
    # colocar tudo o que não é borda a falso
    for i in range(1, N):
        for j in range(1, N):
            if (i,j) not in init_state:
                init_state[(i,j)] = (state['pos'][f'({i},{j})'] == BitVecVal(0,5))
    
      
    # border-x
    for i in range(0,1):
        for j in range(1, N):
            if (i,j) not in init_state:
                init_state[(i,j)]=(state['pos'][f'({i},{j})'] == BitVecVal(int(pr.prob(prob, num=1)), 5))
                
    # border-y
    for i in range(1,N):
        for j in range(0, 1):
            if (i,j) not in init_state:
                init_state[(i,j)]=(state['pos'][f'({i},{j})'] == BitVecVal(int(pr.prob(prob, num=1)), 5))
                
    init_state[(0,0)]=(state['pos'][f'({0},{0})'] == BitVecVal(int(pr.prob(prob, num=1)),5))
                
    statements = list(init_state.values())
    
    print(And(statements))
    return And(
        state['pc'] == 0,
        And(statements)
    )



In [133]:
def vizinhos(i, j, N):
    vizinhos = []
    
    for y in range(i-1, i+2):
        for x in range(j-1, j+2):
            if  0 <= y < N and 0 <= x < N and (y,x) != (i,j):
                vizinhos.append((y,x))
        
    return vizinhos
       

#print(vizinhos(1,1,4))
def trans(curr, prox, N):
    
    rules = And([
               Or(
                   And(
                       sum([curr['pos'][f'({v[0]},{v[1]})'] for v in vizinhos(i,j,N)]) == BitVecVal(3, 5),
                       curr['pos'][f'({0},{0})'] == BitVecVal(0, 5),
                       prox['pos'][f'({0},{0})'] == BitVecVal(1, 5)
                   ),
                   And(
                        And(
                            UGE(sum([curr['pos'][f'({v[0]},{v[1]})'] for v in vizinhos(i,j,N)]), BitVecVal(2, 5)),
                            ULE(sum([curr['pos'][f'({v[0]},{v[1]})'] for v in vizinhos(i,j,N)]), BitVecVal(3, 5))
                          ),
                       curr['pos'][f'({i},{j})'] == BitVecVal(1, 5), 
                       prox['pos'][f'({i},{j})'] == BitVecVal(1, 5)
                   ),
                   And(
                   )
                   
               ) 
            
                for i in range(1,N)
                for j in range(1,N)
            ])
    
    
    stay_border_x = And([ prox['pos'][f'({i},{j})'] == curr['pos'][f'({i},{j})'] for i in range(0,1) for j in range(1,N) ])
    stay_border_y = And([ prox['pos'][f'({i},{j})'] == curr['pos'][f'({i},{j})'] for i in range(1,N) for j in range(0,1) ])
    
    # t0 -> t1
    t0 = And(
        curr['pc'] == 0,
        prox['pc'] == 1,
        
        rules,
        
        prox['pos'][f'({0},{0})'] == curr['pos'][f'({0},{0})'],
        stay_border_x,
        stay_border_y
    )
    
    # t1 -> t1
    t1 = And(
        curr['pc'] == 1,
        prox['pc'] == 1,
       
        #sum([curr['pos'][f'({i},{j})'] for i in range(1,N) for j in range(1,N)]) == BitVecVal(9,5),
        #And([prox['pos'][f'({i},{j})'] == curr['pos'][f'({i},{j})'] for i in range(1,N) for j in range(1,N)]),
        
        UGE(sum([curr['pos'][f'({i},{j})'] for i in range(1,N) for j in range(1,N)]), BitVecVal(1,5)),
        rules,
        
        prox['pos'][f'({0},{0})'] == curr['pos'][f'({0},{0})'],
        stay_border_x,
        stay_border_y
    )

    #t1 -> t2
    t2 = And(
        curr['pc'] == 1,
        prox['pc'] == 2,
        
        sum([curr['pos'][f'({i},{j})'] for i in range(1,N) for j in range(1,N)]) == BitVecVal(0,5),
        
        prox['pos'][f'({0},{0})'] == curr['pos'][f'({0},{0})'],
        stay_border_x,
        stay_border_y,
        
       # sum([curr['pos'][f'({i},{j})'] for i in range(1,N) for j in range(1,N)]) == BitVecVal(0, 5)
    )
    
    #t2 -> t2
    stop = And(
        curr['pc'] == 2,
        prox['pc'] == 2,
        
        And([prox['pos'][f'({i},{j})'] == curr['pos'][f'({i},{j})'] for i in range(0,N) for j in range(0,N)])
    )

    
    return Or(t0, t1, t2, stop)
    

In [134]:
def teste():
    from pyprobs import Probability as pr
    x = pr.prob("00%", num=1)
    return int(x)


    import random

    data = [0,1]
    probability = [0.7, 0.3]
 
    return random.choices(data, probability)

    return 
teste()

0

In [136]:
def gera_traco(declare,init,k, N):

    s = Solver()
    
    trace = [declare(i, N) for i in range(k)]

    # adicionar o estado inicial
    s.add(init(trace[0],"0%",N))
    
    # adicionar as transições
    for i in range(k - 1):
        s.add(trans(trace[i], trace[i+1], N))
    
    
    check = s.check()
    if check == sat:
        m = s.model()
        #print(m)
        for i in range(k):
            for v in trace[i]:
                if v == 'pos':
                    for k in range(0, N):
                        for j in range(0, N):
                            if m[trace[i][v][f'({k},{j})']] == 0:
                                print("F", end=" ")
                            else:
                                print("T", end=" ")
                        print("\n")        
                else:
                    print(v, "=", m[trace[i][v]])
            print("----------------")
    else:
        print(check)
                
gera_traco(declare,init,20,4)

And(1 == pos(1,1)0,
    1 == pos(1,2)0,
    1 == pos(1,3)0,
    1 == pos(2,1)0,
    1 == pos(2,2)0,
    1 == pos(2,3)0,
    1 == pos(3,1)0,
    1 == pos(3,2)0,
    1 == pos(3,3)0,
    0 == pos(0,1)0,
    0 == pos(0,2)0,
    0 == pos(0,3)0,
    0 == pos(1,0)0,
    0 == pos(2,0)0,
    0 == pos(3,0)0,
    0 == pos(0,0)0)
pc = 0
F F F F 

F T T T 

F T T T 

F T T T 

----------------
pc = 1
F F F F 

F T F F 

F F T F 

F T T T 

----------------
pc = 1
F F F F 

F T T F 

F T T T 

F T T T 

----------------
pc = 1
F F F F 

F T T T 

F T T T 

F T T T 

----------------
pc = 1
F F F F 

F T T T 

F T T T 

F T T T 

----------------
pc = 1
F F F F 

F F F T 

F T T T 

F T T T 

----------------
pc = 1
F F F F 

F T T F 

F T T T 

F T T T 

----------------
pc = 1
F F F F 

F T T T 

F T T T 

F T T T 

----------------
pc = 1
F F F F 

F T T T 

F T F T 

F T T T 

----------------
pc = 1
F F F F 

F T F T 

F T T T 

F T F T 

----------------
pc = 1
F F F F 

F F T T 

F T F T 

F T